In [7]:
import torch
import collections # collections 모듈에 있는 deque를 사용하기 위함
#dequq는 double-ended queue로써, queue와 stack의 기능을 다 쓸 수 있다.

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np


In [14]:
class enva():
    def __init__(self):
        self.height = 10
        self.width = 10
        self.num_obstacle_range =5
        self.num_obstacle_range_min = 3
        self.turn = 0
        self.position = int(round(self.width/2))
        self.map = np.array([[0.0 for j in range(self.width)] for i in range(self.height)])
        self.map[self.height-1][self.position] = 2.
        self.done = False
        
    def step(self, key):
        #obstacle down
        i = self.height - 2
        while(i>=0):
            self.map[i+1] = self.map[i]
            i -= 1
        #obstacle init(first_line)
        self.map[0] = np.array([0 for j in range(self.width)])
        #obstacle making
        num_obstacle = random.randrange(self.num_obstacle_range_min,self.num_obstacle_range)
        i = 0
        while(i < num_obstacle):
            position_obstacle = random.randrange(0,self.width)
            if (self.map[0][position_obstacle] == 1.0):
                continue
            self.map[0][position_obstacle] = 1.0
            i += 1
        #big_obstacle
        big_obstacle = random.randrange(0,20)
        if(big_obstacle == -1): ############### 1로 바꾸면 큰 똥이 생긴다.
            big_obstacle = random.randrange(1,self.width-1)
            for j in range(3):
                for k in range(-1,2):
                    self.map[j][big_obstacle + k] = 1.
        #player position
        reward = 1
        # 0 : 왼쪽, 1 : 가만히, 2 : 오른쪽
        if key == 0:
            if(self.position>0):
                self.position -= 1
            
        if key == 1:
            pass      
        if key == 2:
            if(self.position<self.width-1):
                self.position += 1
                  
        if (self.map[self.height-1][self.position] == 1.0):
                    reward = 0
                    self.done = True
                    #print("====Game Over====")
                        
        self.map[self.height-1][self.position] = 2.0
        #reward(turn(time))
        self.turn += 1
        
        return torch.flatten(torch.tensor(self.map),0), reward, self.done, _ 

    def reset(self):
        self.turn = 0
        self.position = round(self.width/2)
        self.map = np.array([[0.0 for j in range(self.width)] for i in range(self.height)])
        self.map[self.height-1][self.position] = 2.0
        self.done = False
        
        return torch.flatten(torch.tensor(self.map),0)

In [15]:
#nerual network setting -> good performance
#hyperparameters
buffer_size =30000   # environment's size에 맞춰서 setting
batch_size = 32   
gamma = 0.98  # Large future weight

class Agent(nn.Module):
    def __init__(self):
        super(Agent,self).__init__()
        self.fc1 = nn.Linear(100,128)
        self.fc2 = nn.Linear(128,128)
        self.fc3 = nn.Linear(128,128)
        self.fc4 = nn.Linear(128,3)  # 좌우, 제자리
  
    def forward(self,x):
        
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x
  
  
    def select_action(self,state,eps):   # state, reward를 받으면 action이 나오는 relation. 
                                        # but reward는 memory를 통해 사용하므로 state를 받아서 action을 뽑는다.

        coin = random.random() # 0~1
        out_value = self.forward(state) # 
        if coin < eps:   # random action
            a = random.randint(0,2) # 0 ,1 ,2 choice
        else : # optimize action
            a = torch.argmax(out_value).item() # a는 tensor가 아닌 int로 들어가야 한다.
                                               # 둘 중 높은 value를 가진 index가 action이다. 
        return a

class replay_buffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen = buffer_size)
    def put(self,input): # buffer에 store
        self.buffer.append(input) # dtype = tensor  4개가 한 묶음으로 들어가야됨

    def sampling(self,bat_ch): 
        mini_batch = random.sample(self.buffer,bat_ch) # bat_ch X 4 (list), dtype = tensor
                                                       # buffer에서 batch_size (현재 32)만큼 sampling을 한다.
        #s_list, a_list, r_prime_list, s_prime_list,done_mask_lst = [],[],[],[],[] # list로 각각을 저장할거임
        a_list, r_prime_list, done_mask_lst = [],[],[]
        s_list = torch.tensor([0 for i in range(100)],dtype = torch.float).unsqueeze(dim=0)
        s_prime_list = torch.tensor([0 for i in range(100)],dtype = torch.float).unsqueeze(dim=0)
        
        for transition in mini_batch:
              s, a, r_prime, s_prime,done_mask = transition
              
              
              # a : int, r_prime : float
              s_list = torch.cat([s_list,s.unsqueeze(dim=0)],dim=0) 
              s_prime_list = torch.cat([s_prime_list,s_prime.unsqueeze(dim=0)],dim=0)         
              #s_list.append(s)
              a_list.append([a]) # 열로 쌓는다
              r_prime_list.append([r_prime])
              #s_prime_list.append(s_prime)
              done_mask_lst.append([done_mask])
        s_list = s_list[1:,:]
        s_prime_list = s_prime_list[1:,:]      
        return torch.tensor(s_list, dtype=torch.float), torch.tensor(a_list), \
                       torch.tensor(r_prime_list), torch.tensor(s_prime_list, dtype=torch.float),torch.tensor(done_mask_lst)

    def size(self):
        return len(self.buffer)

def train(q,q_target,disk,optimizer):
    for i in range(10):
        s, a, r, s_prime,done_mask = disk.sampling(batch_size) # s : batch_size x 4    

        q_out = q(s) # batch_size x 4 
        q_a = q_out.gather(1,a) # dim =1에서 index가 a인 것들로 재구성
        #print(q_a.shape)  # batch_size x 1
        qtarget_out = q_target(s_prime) # s'에서의 value가 4개 나옴 -> batch_size x 4
        #print(qtarget_out.max(1)) 값 반환 하는데, 이 값은 행벡터
        max_q_prime = qtarget_out.max(1)[0].unsqueeze(1) # dim = 1 에서 max값 가지고 온다. batch_size x 1로 만듦

        target = r + max_q_prime*gamma* done_mask 
        loss = F.smooth_l1_loss(q_a,target)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

In [22]:
env = enva()
q = Agent()
q_target = Agent()
q_target.load_state_dict(q.state_dict())
memory = replay_buffer()  
q.load_state_dict(torch.load('DQN_ddong_with_dongmin.pt'))
optimizer = optim.Adam(q.parameters(),lr = 0.0005)
score = 0.0
best_score = 0.

for epi in range(30000):

    eps = max(0.0, 0.08 - 0.01*(epi/200)) #0,08
    s = env.reset()
   
    done = False
    one_score = 0
    while not done :
   
        a = q.select_action(s.float(),eps)
        s_prime,r,done,info = env.step(a)
        done_mask = 0.0 if done else 1.0  # termination이면 done을 0으로..
        memory.put((s,a,r,s_prime, done_mask)) # state, action, reward, state', done_mask
        s = copy.deepcopy(s_prime)
        score += r
        one_score += r
    if one_score > best_score:
        best_score = one_score
        print("...save model...")
        torch.save(q.state_dict(),"DQN_ddong_with_dongmin.pt")
        print("best score : ",best_score)
    if memory.size() >3000 :
        train(q,q_target,memory,optimizer)

    if epi%100==0 and epi!=0:
        q_target.load_state_dict(q.state_dict())
        print("n_episode :{}, score : {:.1f}, n_buffer : {},eps = {}".format(
                                                        epi, score/100, memory.size(),eps))
        
        score = 0.0





best score :  9
best score :  11
best score :  12
best score :  13
best score :  16
best score :  20
best score :  22
n_episode :100, score : 10.7, n_buffer : 1169,eps = 0.075
n_episode :200, score : 10.7, n_buffer : 2337,eps = 0.07


/tmp/ipykernel_1638931/1159536909.py:64: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.tensor(s_list, dtype=torch.float), torch.tensor(a_list), \
/tmp/ipykernel_1638931/1159536909.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  torch.tensor(r_prime_list), torch.tensor(s_prime_list, dtype=torch.float),torch.tensor(done_mask_lst)


n_episode :300, score : 10.9, n_buffer : 3527,eps = 0.065
best score :  24
best score :  27
n_episode :400, score : 12.1, n_buffer : 4833,eps = 0.06
best score :  29
best score :  32
n_episode :500, score : 12.2, n_buffer : 6152,eps = 0.055
best score :  37
n_episode :600, score : 11.5, n_buffer : 7405,eps = 0.05
best score :  38
n_episode :700, score : 13.8, n_buffer : 8884,eps = 0.045
n_episode :800, score : 13.3, n_buffer : 10317,eps = 0.04
n_episode :900, score : 13.6, n_buffer : 11780,eps = 0.035
n_episode :1000, score : 13.7, n_buffer : 13251,eps = 0.03
best score :  39
n_episode :1100, score : 13.7, n_buffer : 14723,eps = 0.025
best score :  41
n_episode :1200, score : 15.3, n_buffer : 16350,eps = 0.020000000000000004
best score :  50
n_episode :1300, score : 14.8, n_buffer : 17926,eps = 0.015
n_episode :1400, score : 15.4, n_buffer : 19570,eps = 0.009999999999999995
n_episode :1500, score : 15.8, n_buffer : 21255,eps = 0.0050000000000000044
best score :  88
n_episode :1600, sco

n_episode :13600, score : 113.9, n_buffer : 30000,eps = 0.0
n_episode :13700, score : 74.1, n_buffer : 30000,eps = 0.0
n_episode :13800, score : 83.3, n_buffer : 30000,eps = 0.0
n_episode :13900, score : 102.0, n_buffer : 30000,eps = 0.0
n_episode :14000, score : 108.1, n_buffer : 30000,eps = 0.0
n_episode :14100, score : 126.3, n_buffer : 30000,eps = 0.0
n_episode :14200, score : 125.9, n_buffer : 30000,eps = 0.0
n_episode :14300, score : 105.7, n_buffer : 30000,eps = 0.0
n_episode :14400, score : 97.2, n_buffer : 30000,eps = 0.0
n_episode :14500, score : 96.8, n_buffer : 30000,eps = 0.0
n_episode :14600, score : 130.6, n_buffer : 30000,eps = 0.0
n_episode :14700, score : 105.5, n_buffer : 30000,eps = 0.0
best score :  1080
n_episode :14800, score : 141.7, n_buffer : 30000,eps = 0.0
n_episode :14900, score : 123.4, n_buffer : 30000,eps = 0.0
n_episode :15000, score : 119.2, n_buffer : 30000,eps = 0.0
n_episode :15100, score : 107.3, n_buffer : 30000,eps = 0.0
n_episode :15200, score :

n_episode :27300, score : 95.5, n_buffer : 30000,eps = 0.0
n_episode :27400, score : 120.2, n_buffer : 30000,eps = 0.0
n_episode :27500, score : 139.7, n_buffer : 30000,eps = 0.0
n_episode :27600, score : 156.0, n_buffer : 30000,eps = 0.0
n_episode :27700, score : 101.3, n_buffer : 30000,eps = 0.0
n_episode :27800, score : 127.1, n_buffer : 30000,eps = 0.0
n_episode :27900, score : 143.7, n_buffer : 30000,eps = 0.0
n_episode :28000, score : 147.2, n_buffer : 30000,eps = 0.0
n_episode :28100, score : 129.1, n_buffer : 30000,eps = 0.0
n_episode :28200, score : 136.6, n_buffer : 30000,eps = 0.0
n_episode :28300, score : 127.4, n_buffer : 30000,eps = 0.0
n_episode :28400, score : 147.7, n_buffer : 30000,eps = 0.0
n_episode :28500, score : 128.8, n_buffer : 30000,eps = 0.0
n_episode :28600, score : 159.6, n_buffer : 30000,eps = 0.0
n_episode :28700, score : 120.7, n_buffer : 30000,eps = 0.0
n_episode :28800, score : 150.4, n_buffer : 30000,eps = 0.0
n_episode :28900, score : 211.9, n_buffer

In [ ]:
import sys
from time import sleep
import numpy as np
import random
import keyboard
import copy

height = 10
width = 10
num_obstacle_range =7
num_obstacle_range_min = 4
turn = 0
position = round(width/2)
player_position = round(width/2)
map = np.array([[0. for j in range(width)] for i in range(height)])
map[height-1][position] = 2.

player_map = copy.deepcopy(map)
q.eval()
with torch.no_grad():

    while True:
        #print_map
        action = q.forward(torch.flatten(torch.tensor(map).float()))
        print("===============ai map==================")
        print(map)
        print("===============player map==================")
        print(player_map)
        print("Turn:" ,turn)
        #obstacle down
        i = height - 2
        while(i>=0):
            map[i+1] = map[i]
            i -= 1
        #obstacle init(first_line)
        map[0] = np.array([0. for j in range(width)])
        #obstacle making
        num_obstacle = random.randrange(num_obstacle_range_min,num_obstacle_range)
        i = 0
        while(i < num_obstacle):
            position_obstacle = random.randrange(0,width)
            if (map[0][position_obstacle] == 1.):
                continue
            map[0][position_obstacle] = 1.
            i += 1
        #big_obstacle
        big_obstacle = random.randrange(0,20)
        if(big_obstacle == 0):
            big_obstacle = random.randrange(1,width-1)
            for j in range(3):
                for k in range(-1,2):
                    map[j][big_obstacle + k] = 1.
        #copy player_map
        player_map = copy.deepcopy(map)
        #ai position
        #print("value : ",action)
        action = torch.argmax(action).item()
        #print("action : ", action)
        
        while True:
            player_key = input()
            key = action
            # player action
            if player_key == "j":
                if(player_position>0):
                    player_position -= 1
                
            if player_key == "k":
                pass        
            if player_key == "l":
                if(player_position<width-1):
                    player_position += 1
                        

            # ai action
            if key == 0:
                if(position>0):
                    position -= 1
                break
            if key == 1:
                break        
            if key ==2:
                if(position<width-1):
                    position += 1
                break        
        if map[height-1][position] == 1 or player_map[height-1][player_position] == 1:
                    print("====Game Over====")
                    if map[height-1][position] == 1:
                        print("player win!!\n\n")
                    
                    elif player_map[height-1][player_position] == 1:
                        print("AI win!!\n\n")
                    sys.exit()    
        map[height-1][position] = 2.
        player_map[height-1][player_position] = 2.
        #reward(turn(time))
        turn += 1
        
